In [4]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
print(torch.cuda.device_count())  # print 1

import numpy as np

import sys
from pathlib import Path

sys.path.append('/home/nikita/e2e-driving')

import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torch.distributions import Categorical
import math

from ebm import optimizers
from dataloading.nvidia import NvidiaValidationDataset
from metrics.metrics import calculate_open_loop_metrics
from pilotnet import PilotnetEBM
import pandas as pd


1


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@torch.no_grad()
def evaluate(model, dataloader, dfo_reorder=False):
    model.eval()
    all_predictions = []
    inference_times = []
    progress_bar = tqdm(total=len(dataloader), smoothing=0)

    epoch_mae = 0.0
    epoch_entropy = 0.0
    ask_batch_timestamp = time.time()
    for i, (input, target, _) in enumerate(dataloader):
        recv_batch_timestap = time.time()

        inputs = input['image'].to(device)
        target = target.to(device, torch.float32)

        inference_start = time.perf_counter()
        preds, energy = model(inputs, dfo_reorder=False)
        inference_end = time.perf_counter()

        inference_time = inference_end - inference_start
        inference_times.append(inference_time)

        mae = F.l1_loss(preds, target.view(-1, 1))
        mae_degrees = math.degrees(mae.item())
        epoch_mae += mae_degrees

        entropy = Categorical(F.softmax(-energy, dim=-1)).entropy().mean() / math.log(energy.shape[-1])
        epoch_entropy += entropy.item()

        all_predictions.extend(preds.cpu().squeeze().numpy())

        progress_bar.update(1)
        progress_bar.set_description(f'MAE: {(epoch_mae / (i + 1)):.4f} | Entropy: {(epoch_entropy / (i + 1)):.4f}')

        ask_batch_timestamp = time.time()

    avg_mae = epoch_mae / len(dataloader)
    avg_entropy = epoch_entropy / len(dataloader)
    result = np.array(all_predictions)
    return avg_mae, avg_entropy, result


def calculate_metrics(fps, predictions, valid_loader):
    '''For steering angle only.'''

    frames_df = valid_loader.dataset.frames
    true_steering_angles = frames_df.steering_angle.to_numpy()
    metrics = calculate_open_loop_metrics(predictions, true_steering_angles, fps=fps)
    return metrics

In [6]:
dataset_path = '/data/Bolt/dataset-new-small/summer2021'
batch_size = 256
num_workers = 8

validset = NvidiaValidationDataset(Path(dataset_path), group_size=1)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=False,
                                          num_workers=num_workers, pin_memory=True,
                                          persistent_workers=True)

/data/Bolt/dataset-new-small/summer2021/2021-05-28-15-19-48_e2e_sulaoja_20_30: length=10708, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: length=25836, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: length=3003, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: length=4551, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: length=25368, filtered=1
/data/Bolt/dataset-new-small/summer2021/2021-09-24-14-03-45_e2e_rec_ss11_backwards: length=25172, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: length=33045, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: length=33281, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-20-15-11-29_e2e_rec_vastse_ss13_17_back: length=26763, filtered=0
/data/Bolt/dataset-new-small/summer2021/2021-10-11-14-50-59_e2e_rec_va

In [7]:
pt_model_path = '/home/nikita/e2e-driving/models/20220618185552_steering-angle/last.pt'

def evaluate_model(constant_actions=False, dfo_reorder=False, **kwargs):
  model = PilotnetEBM()
  inference_model = optimizers.DFOptimizerConst if constant_actions else optimizers.DFOptimizer
  model = inference_model(model, optimizers.DerivativeFreeConfig(**kwargs))
  model.load_state_dict(torch.load(pt_model_path))
  model.to(device)
  mae, entropy, preds = evaluate(model, valid_loader, dfo_reorder=dfo_reorder)

  return mae, entropy, preds

In [8]:
inputs = iter(valid_loader).next()[0]['image'][0].unsqueeze(0).to(device)
print(inputs.shape)

torch.Size([1, 3, 68, 264])


In [11]:
import itertools
import json
import wandb

fps = 30

samples = [1024]
iters = [3, 0]
constant_actions = [False]
dfo_reorders = [True, False]

run_hparams = list(itertools.product(samples, iters, constant_actions, dfo_reorders))

for idx, (samples, iters, constant_actions, dfo_reorder) in enumerate(run_hparams):
    wandb.init(project="ibc-tuning", name=f'{samples}s{iters}it', config={"samples": samples, "iters": iters, "model_path": pt_model_path, "constant_actions": constant_actions, "dfo_reorder": dfo_reorder})

    print(f'running experiment {idx+1}/{len(run_hparams)}. samples: {samples}, iters: {iters}')
    mae, entropy, preds = evaluate_model(constant_actions=constant_actions, dfo_reorder=dfo_reorder, inference_samples=samples, iters=iters)
    metrics = calculate_metrics(fps, preds, valid_loader)
    metrics["entropy"] = entropy
    wandb.log(metrics)
    print('experiment metrics:')
    print(json.dumps(str(metrics), indent=2))
    print()
    wandb.finish()

running experiment 1/4. samples: 1024, iters: 3


  0%|          | 0/869 [00:00<?, ?it/s]

experiment metrics:
"{'mae': 8.043990276034657, 'rmse': 27.661250920183427, 'max': 718.4823656119165, 'whiteness': 384.7091, 'expert_whiteness': 25.61257234059799, 'entropy': 0.9686547157679384, '_timestamp': 1659955012, '_runtime': 981}"



entropy,▁
expert_whiteness,▁
mae,▁
max,▁
rmse,▁
whiteness,▁
entropy,0.96865
expert_whiteness,25.61257
mae,8.04399
max,718.48237
rmse,27.66125


running experiment 2/4. samples: 1024, iters: 3


...

entropy,▁
expert_whiteness,▁
mae,▁
max,▁
rmse,▁
whiteness,▁
entropy,0.55824
expert_whiteness,25.61257
mae,8.02233
max,717.74238
rmse,27.67892


running experiment 4/4. samples: 1024, iters: 0


  0%|          | 0/869 [00:00<?, ?it/s]

experiment metrics:
"{'mae': 8.01759233223066, 'rmse': 27.647336869973067, 'max': 718.3598985709009, 'whiteness': 383.9023, 'expert_whiteness': 25.61257234059799, 'entropy': 0.5581876525464623, '_timestamp': 1659955942, '_runtime': 143}"



entropy,▁
expert_whiteness,▁
mae,▁
max,▁
rmse,▁
whiteness,▁
entropy,0.55819
expert_whiteness,25.61257
mae,8.01759
max,718.3599
rmse,27.64734
